In [32]:
import ee
import geemap
import folium

# เริ่มต้น Google Earth Engine
ee.Initialize()

polygon = ee.FeatureCollection("users/sakdahomhuan/udfire_site");
site = polygon.geometry().bounds()

def get_min(img, geom):
    min_val = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=geom,
        scale=100,
        maxPixels=1e9
    ).getInfo()

    print(min_val)
    return min_val['constant']

def get_max(img, geom):
    max_val = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=geom,
        scale=100,
        maxPixels=1e9
    ).getInfo()
    print(max_val)
    return max_val['constant']

def calculateNBR(image):
    nir = image.select('B8')
    swir = image.select('B12')
    nbr = nir.subtract(swir).divide(nir.add(swir)).rename('NBR')
    return image.addBands(nbr)

def calculateCSI(image):
    nir = image.select('B8')
    swir = image.select('B12')
    csi = nir.divide(swir).rename('CSI')
    return image.addBands(csi)

def prepareData(image, geometry):
    return image.clip(geometry)

# โหลดข้อมูล Sentinel-2 และกรองตามช่วงเวลา
def loadSentinel2Data(dateRange, geometry):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(geometry) \
                    .filterDate(dateRange) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
                    .map(lambda image: image.clip(geometry)) \
                    .map(calculateNBR) \
                    .map(calculateCSI)
    return collection.median()

def showMap(feat, geom, param):
  empty = ee.Image().byte()
  imageUsc = empty.paint( featureCollection=feat, color=param)
  visualization = {
      'min': get_min(imageUsc, geom),
      'max': get_max(imageUsc, geom),
      'palette': ['green', 'yellow', 'red']
  }

def get_image_url(image, vis_params, name, m):
  map_id_dict = ee.Image(image).getMapId(vis_params)
  folium.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True,
      opacity=0.8
  ).add_to(m)

def export_to_drive(image, name, folder, geometry):
  export_task = ee.batch.Export.image.toDrive(**{
      'image': image,
      'description': name,
      'folder': folder,
      'scale': 20,
      'region': geometry,
      'fileFormat': 'GeoTIFF'
  })
  export_task.start()

In [40]:
geometry = site
preFireDateRange = ee.DateRange('2023-10-01', '2023-12-31')
postFireDateRange = ee.DateRange('2024-02-01', '2024-05-31')

preFireImage = loadSentinel2Data(preFireDateRange, geometry)
postFireImage = loadSentinel2Data(postFireDateRange, geometry)

# คำนวณ dNBR dCSI
dNBR = postFireImage.select('NBR').subtract(preFireImage.select('NBR')).rename('dNBR')
dCSI = postFireImage.select('CSI').subtract(preFireImage.select('CSI')).rename('dCSI')

# Create map
map_center = [17.6694, 100.4609]
f = folium.Figure(height=600)
m = folium.Map(location=map_center, zoom_start=11).add_to(f)

# google satellite
google_satellite = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    overlay=False,
    control=True,
    show=False
).add_to(m)

#  google terrain
google_terrain = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Terrain',
    overlay=False,
    control=True,
    show=True
).add_to(m)

vis_truecolor =  {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
vis_params_br = {'min': -1, 'max': 1, 'palette': ['white', 'black', 'red']} 

preFireImage_url = get_image_url(image=preFireImage, vis_params=vis_truecolor, name='Pre-fire Image', m=m)
postFireImage_url = get_image_url(image=postFireImage, vis_params=vis_truecolor, name='Post-fire Image', m=m)
dNBR_url = get_image_url(image=dNBR, vis_params=vis_params_br, name='dNBR', m=m)
dCSI_url = get_image_url(image=dCSI, vis_params=vis_params_br, name='dCSI', m=m)

folium.LayerControl().add_to(m)


export_to_drive(image=dCSI, name='dCSI_2024', folder='_GEE_TUM', geometry=geometry)

m